## Notebook to analyze MD and experimental S2 values for Ribonuclease H

Notebook used for Figures 5, 6, and 7 and associated calculations in the paper:

Parsing Dynamics of Protein Backbone NH and Side-Chain Methyl Groups using Molecular Dynamics Simulations

Nooriel E. Banayan, Andrew Hsu, John F. Hunt, Arthur G. Palmer, III, and Richard A. Friesner

J. Chem. Theory Comput. (2024). https://doi.org/10.1021/acs.jctc.4c00378.

The notebook analyzes $\chi$ angle distributions to determine generalized order parameters.

Dihedral angles were measured using 'nmr_orderD' written by K. A. Sharp

--------------------------------------------
MIT License

Copyright (c) 2024 Arthur G. Palmer, III

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.


In [ ]:
import numpy as np

import matplotlib
#print(matplotlib.__version__)
from matplotlib import pyplot as plt
import matplotlib.animation as animation
from matplotlib import rc

from scipy import linalg
from scipy.optimize import curve_fit
from scipy.optimize import brentq
from scipy.special import erf
from scipy.special import erfi

import re

from IPython.display import HTML

%matplotlib widget


In [ ]:
# eye-friendly colors from https://jfly.uni-koeln.de/color/

ReddishPurple = [204/255.0,121/255.0,167/255.0]
Orange = [230/255.0,159/255.0,0/255.0]
Green = [0,158/255.0,115/255.0]
Blue = [0,114/255.0,178/255.0]

In [ ]:
def modY2q(q,theta,phi):
    y = 0
    if q == 0:
        y = 0.5*(3*np.cos(theta)**2-1)
    elif q == 1:
        y = -np.sqrt(3.0/2.0)*np.cos(theta)*np.sin(theta)
        y = y*np.exp(1j*phi)
    elif q == -1:
        y = -np.sqrt(3.0/2.0)*np.cos(theta)*np.sin(theta)
        y = y*np.exp(1j*phi)
        y = -np.conj(y)
    elif q == 2:
        y = np.sqrt(3.0/8.0)*np.sin(theta)**2
        y = y*np.exp(1j*2*phi)
    elif q == -2:
        y = np.sqrt(3.0/8.0)*np.sin(theta)**2
        y = y*np.exp(1j*2*phi)
        y = np.conj(y)
    return y

def modY2qAll(theta,phi):
    y0 = 0.5*(3*np.cos(theta)**2-1)
    if isinstance(phi,np.ndarray):
        y0 = y0*np.ones(len(phi))

    y = np.sqrt(3.0/2.0)*np.cos(theta)*np.sin(theta)
    y1 = -y*np.exp(1j*phi)
    ym1 = y*np.exp(-1j*phi)
    y = np.sqrt(3.0/8.0)*np.sin(theta)**2
    y2 = y*np.exp(1j*2*phi)
    ym2 = y*np.exp(-1j*2*phi)
    y2all = np.array([ym2,ym1,y0,y1,y2])
    return y2all

def DqqAll(alpha,beta,gamma):
    
    d22 = (1 + np.cos(beta))**2/4.0
    d21 = -np.sin(beta)*(1 + np.cos(beta))/2.0
    d20 = np.sqrt(3.0/8.0)*np.sin(beta)**2
    d2m1 = -np.sin(beta)*(1 - np.cos(beta))/2.0
    d2m2 = (1 - np.cos(beta))**2/4.0
    
    d12 = -d21
    d11 = (2.0*np.cos(beta)**2 + np.cos(beta)-1)/2.0
    d10 = -np.sqrt(3.0/8.0)*np.sin(2*beta)
    d1m1 = (-2*np.cos(beta)**2 + np.cos(beta)+1)/2.0
    d1m2 = d2m1
    
    d0m2 = d20
    d0m1 = d10
    d00 = (3*np.cos(beta)**2-1)/2.0
    d01 = -d10
    d02 = d20
    
    dm1m2 = d21
    dm1m1 = d11
    dm10 = d01
    dm11 = d1m1
    dm12 = -d2m1
    
    dm2m2 = d22 
    dm2m1 = -dm1m2
    dm20 = d02
    dm21 = dm12
    dm22 = d2m2
    
    dmat = np.zeros([5,5],dtype='complex64')
    
    dmat[0,0] = dm2m2
    dmat[0,1] = dm2m1
    dmat[0,2] = dm20
    dmat[0,3] = dm21
    dmat[0,4] = dm22
    
    dmat[1,0] = dm1m2
    dmat[1,1] = dm1m1
    dmat[1,2] = dm10
    dmat[1,3] = dm11
    dmat[1,4] = dm12
    
    dmat[2,0] = d0m2
    dmat[2,1] = d0m1
    dmat[2,2] = d00
    dmat[2,3] = d01
    dmat[2,4] = d02
    
    dmat[3,0] = d1m2
    dmat[3,1] = d1m1
    dmat[3,2] = d10
    dmat[3,3] = d11
    dmat[3,4] = d12
    
    dmat[4,0] = d2m2
    dmat[4,1] = d2m1
    dmat[4,2] = d20
    dmat[4,3] = d21
    dmat[4,4] = d22
        
    for qp in range(-2,3):
        for q in range(-2,3):
            expqp = np.exp(-1j*qp*alpha)
            expq = np.exp(-1j*q*gamma)
            
            dmat[qp+2,q+2] = expqp*dmat[qp+2,q+2]*expq
    
    return dmat

def getChi(dihedArray):
    # dihedArray is array of dihedral angles (in range 0-360)
    
    phi = dihedArray*np.pi/180
    
    chistates = np.zeros(len(dihedArray),dtype=int)
    
    for k,phiK in enumerate(phi):
        if phiK < 0:
            phiK = phiK + 2*np.pi
        
        if phiK >= 0.0 and phiK < 2*np.pi/3:
            chistates[k] = 1
        elif phiK >= 2*np.pi/3 and phiK < 4*np.pi/3:
            chistates[k] = 0
        elif phiK >= 4*np.pi/3 and phiK < 2*np.pi:
            chistates[k] = -1

    return chistates

def getChi1Chi2(chi1,chi2):
    # chi1 and chi2 are arrays of dihedral angles (in range 0-360)
    
    phi1 = chi1*np.pi/180
    phi2 = chi2*np.pi/180
    
    chistates12 = np.zeros(len(phi1),dtype=int)
    chipops = np.zeros(9)
    chimeans = np.zeros([9,2])
    
    for k,phiK1 in enumerate(phi1):
        phiK2 = phi2[k]
        
        if phiK1 < 0:
            phiK1 = phiK1 + 2*np.pi
        if phiK2 < 0:
            phiK2 = phiK2 + 2*np.pi
            
        if phiK1 >= 0.0 and phiK1 < 2*np.pi/3:
            if phiK2 >= 0.0 and phiK2 < 2*np.pi/3:            # pp
                chistates12[k] = 4
                chipops[8]  = chipops[8] + 1 
                chimeans[8,0] = chimeans[8,0] + phiK1
                chimeans[8,1] = chimeans[8,1] + phiK2
            elif phiK2 >= 2*np.pi/3 and phiK2 < 4*np.pi/3:    # pt
                chistates12[k] = 3
                chipops[7]  = chipops[7] + 1
                chimeans[7,0] = chimeans[7,0] + phiK1
                chimeans[7,1] = chimeans[7,1] + phiK2
            elif phiK2 >= 4*np.pi/3 and phiK2 < 2*np.pi:      # pm
                chistates12[k] = 2
                chipops[6]  = chipops[6] + 1
                chimeans[6,0] = chimeans[6,0] + phiK1
                chimeans[6,1] = chimeans[6,1] + phiK2
        
        elif phiK1 >= 2*np.pi/3 and phiK1 < 4*np.pi/3:
            if phiK2 >= 0.0 and phiK2 < 2*np.pi/3:            # tp
                chistates12[k] = 1
                chipops[5]  = chipops[5] + 1
                chimeans[5,0] = chimeans[5,0] + phiK1
                chimeans[5,1] = chimeans[5,1] + phiK2
            elif phiK2 >= 2*np.pi/3 and phiK2 < 4*np.pi/3:    # tt
                chistates12[k] = 0
                chipops[4]  = chipops[4] + 1
                chimeans[4,0] = chimeans[4,0] + phiK1
                chimeans[4,1] = chimeans[4,1] + phiK2
            elif phiK2 >= 4*np.pi/3 and phiK2 < 2*np.pi:      # tm
                chistates12[k] = -1
                chipops[3]  = chipops[3] + 1
                chimeans[3,0] = chimeans[3,0] + phiK1
                chimeans[3,1] = chimeans[3,1] + phiK2
        
        elif phiK1 >= 4*np.pi/3 and phiK1 < 2*np.pi:
            if phiK2 >= 0.0 and phiK2 < 2*np.pi/3:            # mp
                chistates12[k] = -2
                chipops[2]  = chipops[2] + 1
                chimeans[2,0] = chimeans[2,0] + phiK1
                chimeans[2,1] = chimeans[2,1] + phiK2
            elif phiK2 >= 2*np.pi/3 and phiK2 < 4*np.pi/3:    # mt
                chistates12[k] = -3
                chipops[1]  = chipops[1] + 1
                chimeans[1,0] = chimeans[1,0] + phiK1
                chimeans[1,1] = chimeans[1,1] + phiK2
            elif phiK2 >= 4*np.pi/3 and phiK2 < 2*np.pi:      # mm
                chistates12[k] = -4
                chipops[0]  = chipops[0] + 1
                chimeans[0,0] = chimeans[0,0] + phiK1
                chimeans[0,1] = chimeans[0,1] + phiK2
                
    for i in range(0,9):
        chimeans[i,:] = (180/np.pi)*chimeans[i,:]/np.maximum(1,chipops[i])
        chipops[i] = chipops[i]/len(phi1)
        
    return chipops,chimeans,chistates12

def jumpS2rotamer(popArray,theta=70.5):
    # popArray is np.array([p_gminus,p_trans,p_gplus])
    d1 = 0.0
    d2 = 0.0
    s2 = 0.0
    
    phi = np.pi*2.0/3.0
    
    theta = np.pi*70.5/180.0
    
    for k,popK in enumerate(popArray):
        d1 = d1 + popK**2
        for kp in range(k+1,3):
            d2 = d2 + 2*popK*popArray[kp]
            s2 = s2 + 2*popK*popArray[kp]*np.cos(phi)
            
    f0 = d1 + d2
    f1 = d1 + s2
    
    ytmp = modY2qAll(theta,0)
    s2 = f0*ytmp[2]**2 + 2*f1*ytmp[3]**2 + 2*f1*ytmp[4]**2
    
    return np.real(s2)
        
def jumpS2Chi1(dihedArray,theta=70.5):
    # dihedArray is array of dihedral angles (in range 0-360)
    
    phi = dihedArray*np.pi/180
    
    theta = np.pi*70.5/180.0
    
    numM = 0
    numT = 0
    numP = 0
    
    y2M = np.zeros(5,dtype=np.complex64)
    y2T = np.zeros(5,dtype=np.complex64)
    y2P = np.zeros(5,dtype=np.complex64)
    y2iso = np.zeros(5,dtype=np.complex64)
    
    for k,phiK in enumerate(phi):
        if phiK < 0:
            phiK = phiK + 2*np.pi
        
        ytmp = modY2qAll(theta,phiK)
        for n,yval in enumerate(ytmp):
            y2iso[n] = y2iso[n] + yval
            if phiK >= 0.0 and phiK < 2*np.pi/3:
                y2P[n] = y2P[n] + yval
                numP = numP + 1
            elif phiK >= 2*np.pi/3 and phiK < 4*np.pi/3:
                y2T[n] = y2T[n] + yval
                numT = numT + 1
            elif phiK >= 4*np.pi/3 and phiK < 2*np.pi:
                y2M[n] = y2M[n] + yval
                numM = numM + 1

    numM = numM/5.0
    numT = numT/5.0
    numP = numP/5.0
    
    y2M = y2M/max(1.0,numM)
    y2T = y2T/max(1.0,numT)
    y2P = y2P/max(1.0,numP)
    y2iso = y2iso/len(phi)
    
    s2M = np.real(np.sum(y2M*np.conj(y2M)))
    s2P = np.real(np.sum(y2P*np.conj(y2P)))
    s2T = np.real(np.sum(y2T*np.conj(y2T)))
    
    s2iso = np.real(np.sum(y2iso*np.conj(y2iso)))
    
    pM = float(numM)/len(phi)
    pT = float(numT)/len(phi)
    pP = float(numP)/len(phi)
    
    s2rotamer = pM*s2M + pT*s2T + pP*s2P
    
    popchi1 = np.array([pM,pT,pP])
    
    s2con = jumpS2rotamer(popchi1)
    
    return popchi1,s2rotamer,s2iso,s2con

def jumpS2Chi1Chi2(chi1,chi2,theta=70.5):
    # chi1 and chi2 are arrays of dihedral angles (in range 0-360)
    
    phi1 = chi1*np.pi/180
    phi2 = chi2*np.pi/180
    
    theta = np.pi*70.5/180.0
    
    phi1con = np.zeros(len(phi1))
    phi2con = np.zeros(len(phi2))
    
    popArray = np.zeros([9,5],dtype=np.complex64)
    popNum = np.zeros(9)
    
    y2iso = np.zeros(5,dtype=np.complex64)
    y2con = np.zeros(5,dtype=np.complex64)
    
    # fix negative phi values and find consensus rotamers
    for k,phik in enumerate(phi1):
        if phik < 0:
            phi1[k] = phik + 2*np.pi
            
        if phik >= 0.0 and phik < 2*np.pi/3:
            phi1con[k] = np.pi/3
        elif phik >= 2*np.pi/3 and phik < 4*np.pi/3:
            phi1con[k] = np.pi
        elif phik >= 4*np.pi/3 and phik < 2*np.pi:
            phi1con[k] = 5*np.pi/3
            
    for k,phik in enumerate(phi2):
        if phik < 0:
            phi2[k] = phik + 2*np.pi
            
        if phik >= 0.0 and phik < 2*np.pi/3:
            phi2con[k] = np.pi/3
        elif phik >= 2*np.pi/3 and phik < 4*np.pi/3:
            phi2con[k] = np.pi
        elif phik >= 4*np.pi/3 and phik < 2*np.pi:
            phi2con[k] = 5*np.pi/3
    
    for k,phi2k in enumerate(phi2):
        
        # transform phi2 to phi1 reference frame
        phi1k = phi1[k]
        ytmp = modY2qAll(theta,phi2k)
        dtmp = DqqAll(-phi1k,theta,0)
        
        Y2m2 = dtmp[0,:] @ ytmp
        Y2m1 = dtmp[1,:] @ ytmp
        Y20 = dtmp[2,:] @ ytmp
        Y21 = dtmp[3,:] @ ytmp
        Y22 = dtmp[4,:] @ ytmp
        
        ytrans = np.array([Y2m2,Y2m1,Y20,Y21,Y22],dtype=np.complex64)
        
        # transform consensus phi2 to consensus phi1 reference frame
        phi1conk = phi1con[k]
        phi2conk = phi2con[k]
        ytmpcon = modY2qAll(theta,phi2conk)
        dtmpcon = DqqAll(-phi1conk,theta,0)
        
        Y2m2 = dtmpcon[0,:] @ ytmpcon
        Y2m1 = dtmpcon[1,:] @ ytmpcon
        Y20 = dtmpcon[2,:] @ ytmpcon
        Y21 = dtmpcon[3,:] @ ytmpcon
        Y22 = dtmpcon[4,:] @ ytmpcon
        
        ytranscon = np.array([Y2m2,Y2m1,Y20,Y21,Y22],dtype=np.complex64)
        
        # accumulate sums
        for n in range(0,5):
            
            y2iso[n] = y2iso[n] + ytrans[n]
            y2con[n] = y2con[n] + ytranscon[n]
            
            if phi1k >= 0.0 and phi1k < 2*np.pi/3:
                if phi2k >= 0.0 and phi2k < 2*np.pi/3:
                    popArray[8,n] = popArray[8,n] + ytrans[n]
                    popNum[8] = popNum[8] + 1
                elif phi2k >= 2*np.pi/3 and phi2k < 4*np.pi/3:
                    popArray[7,n] = popArray[7,n] + ytrans[n]
                    popNum[7] = popNum[7] + 1
                elif phi2k >= 4*np.pi/3 and phi2k <= 2*np.pi:
                    popArray[6,n] = popArray[6,n] + ytrans[n]
                    popNum[6] = popNum[6] + 1
                else:
                    print(k,phi1k,phi2k)
            elif phi1k >= 2*np.pi/3 and phi1k < 4*np.pi/3:
                if phi2k >= 0.0 and phi2k < 2*np.pi/3:
                    popArray[5,n] = popArray[5,n] + ytrans[n]
                    popNum[5] = popNum[5] + 1
                elif phi2k >= 2*np.pi/3 and phi2k < 4*np.pi/3:
                    popArray[4,n] = popArray[4,n] + ytrans[n]
                    popNum[4] = popNum[4] + 1
                elif phi2k >= 4*np.pi/3 and phi2k <= 2*np.pi:
                    popArray[3,n] = popArray[3,n] + ytrans[n]
                    popNum[3] = popNum[3] + 1
                else:
                    print(k,phi1k,phi2k)
            elif phi1k >= 4*np.pi/3 and phi1k <= 2*np.pi:
                if phi2k >= 0.0 and phi2k < 2*np.pi/3:
                    popArray[2,n] = popArray[2,n] + ytrans[n]
                    popNum[2] = popNum[2] + 1
                elif phi2k >= 2*np.pi/3 and phi2k < 4*np.pi/3:
                    popArray[1,n] = popArray[1,n] + ytrans[n]
                    popNum[1] = popNum[1] + 1
                elif phi2k >= 4*np.pi/3 and phi2k <= 2*np.pi:
                    popArray[0,n] = popArray[0,n] + ytrans[n]
                    popNum[0] = popNum[0] + 1
                else:
                    print(k,phi1k,phi2k)
            else:
                print(k,phi1k,phi2k)

    y2iso = y2iso/len(phi1)
    y2con = y2con/len(phi1)
    
    s2iso = np.real(np.sum(y2iso*np.conj(y2iso)))
    s2con = np.real(np.sum(y2con*np.conj(y2con)))
    
    poptemp = np.maximum(np.ones(9),popNum/5)
    s2chi12 = np.real(np.sum(popArray*np.conj(popArray),axis=1))
    s2chi12 = s2chi12/poptemp**2
    popchi12 = popNum/(5*len(phi1))
    
    s2rotamer12 = np.sum(popchi12*s2chi12)
    
    return popchi12,s2rotamer12,s2iso,s2con

In [ ]:
# Dihedral angle auto correlation function

def AutoCorr(phiMD,corrdim,theta):
    """Autocorrelation function of input trajectory by brute force calculation.
    phiMD           array of dihedral angles in radians scaled to 0 to 2*pi
    corrdim         length of correlation function
    theta           polar angle for tetrahedral geometry
    """

    y20acf = np.zeros(corrdim,dtype=np.complex64)
    y2m1acf = np.zeros(corrdim,dtype=np.complex64)
    y21acf = np.zeros(corrdim,dtype=np.complex64)
    y2m2acf = np.zeros(corrdim,dtype=np.complex64)
    y22acf = np.zeros(corrdim,dtype=np.complex64)
    ACF = np.zeros(corrdim,dtype=np.complex64)
    
    nsteps = phiMD.shape[0]

    nsamples=nsteps-np.linspace(0,nsteps,nsteps)
    nsamples=nsamples.astype(int)
    
    y2m2,y2m1,y20,y21,y22 = modY2qAll(theta,phiMD)
    
    for i in range(0,corrdim):
           
        y20acf[i] = np.dot(y20[0:nsamples[i]],y20[i:i+nsamples[i]])
        y2m1acf[i] = np.vdot(y2m1[0:nsamples[i]],y2m1[i:i+nsamples[i]])
        y21acf[i] = np.vdot(y21[0:nsamples[i]],y21[i:i+nsamples[i]])
        y2m2acf[i] = np.vdot(y2m2[0:nsamples[i]],y2m2[i:i+nsamples[i]])
        y22acf[i] = np.vdot(y22[0:nsamples[i]],y22[i:i+nsamples[i]])
            
    ACF = y20acf+y2m1acf+y21acf+y2m2acf+y22acf
    
    ACF = ACF/nsamples[0:corrdim]
    
    return np.real(ACF)

def DihedralCrossCorr(phiMD1,phiMD2,corrdim):
    """Cross correlation function of input trajectory by brute force calculation.
    phiMD1          array of dihedral angles in radians scaled to 0 to 2*pi
    phiMD2          array of dihedral angles in radians scaled to 0 to 2*pi
    corrdim         length of correlation function
    """

    cos1 = np.cos(phiMD1)
    sin1 = np.sin(phiMD1)

    cos2 = np.cos(phiMD2)
    sin2 = np.sin(phiMD2)
    
    nsteps = phiMD1.shape[0]
    nsamples=nsteps-np.linspace(0,nsteps,nsteps)
    nsamples=nsamples.astype(int)
    ACF = np.zeros(corrdim)
    
    for i in range(0,corrdim):
           
        ACF[i] = np.mean(cos1[0:nsamples[i]]*cos2[i:i+nsamples[i]]) + np.mean(sin1[0:nsamples[i]]*sin2[i:i+nsamples[i]]) 
    
    return ACF


## Read dihedral angles from Kim Sharp output

In [ ]:
# Read chi data 

infile=open('./2RN2-OPLS4-2us-NVT-allframes_rotamer_torsion_angles.csv','r')

RNHpointer = np.array([],dtype=int)

AA = [ "ILE","LEU", "MET", "THR", "VAL"]

# read once to find number of lines
Nrows = 0
for line in infile:
    line0=line.strip()
    
    if len(line0) > 0 and not re.match("#",line0):
        tmpdata=line0.split(',')
        if tmpdata[0] == "Frame":
            for i,val in enumerate(tmpdata):
                if val[0:3] in AA:
                    RNHpointer = np.append(RNHpointer,int(tmpdata.index(val)))
            RNHchiname = np.array([tmpdata])[0,:][RNHpointer]
        else:
            Nrows = Nrows + 1
            
Nchi = len(RNHpointer)
print(Nrows,Nchi,"\n")
RNHchiOpls4 = np.zeros([Nrows,Nchi])
infile.close()

infile=open('./2RN2-OPLS4-2us-NVT-allframes_rotamer_torsion_angles.csv','r')

j=-1
for line in infile:
    line0=line.strip()
    if len(line0) > 0 and not re.match("Frame",line0):
        tmpdata=line0.split(',')
        j = j + 1    
        if float(tmpdata[0])%10000 == 0:
            print(float(tmpdata[0]))
        RNHchiOpls4[j,:] = np.array([tmpdata])[0,:][RNHpointer]
        
infile.close()

In [ ]:

# Read OPLS4 CH3 order parameters
ch3file=open('./OPLS4_122623/EcRNH_CH3_S2_opls4_tidy.csv','r')

RNHidCH3opls4 = []
RNHresCH3opls4 = []
RNHnumCH3opls4 = np.array([],dtype=int)
RNHtypeCH3opls4 = []
RNHS2CH3opls4 = np.array([])
RNHS2errCH3opls4 = np.array([])


for line in ch3file:
    line0=line.strip()
    
    if len(line0) > 0 and not re.match("Residue",line0):
        tmpdata=line0.split(',')
        RNHnumCH3opls4=np.append(RNHnumCH3opls4,int(tmpdata[0]))
        RNHresCH3opls4=np.append(RNHresCH3opls4,tmpdata[1])
        RNHtypeCH3opls4=np.append(RNHtypeCH3opls4,tmpdata[2])
        RNHidCH3opls4=np.append(RNHidCH3opls4,tmpdata[1][0]+tmpdata[0]+tmpdata[2])
        RNHS2CH3opls4=np.append(RNHS2CH3opls4,float(tmpdata[3]))
        RNHS2errCH3opls4=np.append(RNHS2errCH3opls4,float(tmpdata[4]))

ch3file.close()

RNHS2errCH3opls4 = RNHS2errCH3opls4/np.sqrt(40)

# read OPLS4 CA order parameters
cafile=open('./OPLS4_122623/EcRNH_CA_S2_OPLS4_tidy.csv','r')

RNHidCAopls4 = []
RNHresCAopls4 = []
RNHnumCAopls4 = np.array([],dtype=int)
RNHS2CAopls4 = np.array([])
RNHS2errCAopls4 = np.array([])

AA = ["ALA", "ILE", "LEU", "MET", "THR", "VAL"]
for line in cafile:
    line0=line.strip()
    
    if len(line0) > 0 and not re.match("Residue",line0):
        tmpdata=line0.split(',')
        
        if tmpdata[1] in AA:
            RNHidCAopls4=np.append(RNHidCAopls4,tmpdata[1][0]+tmpdata[0])
            RNHnumCAopls4=np.append(RNHnumCAopls4,int(tmpdata[0]))
            RNHresCAopls4=np.append(RNHresCAopls4,tmpdata[1])
            RNHS2CAopls4=np.append(RNHS2CAopls4,float(tmpdata[2]))
            RNHS2errCAopls4=np.append(RNHS2errCAopls4,float(tmpdata[3]))

cafile.close()

RNHS2errCAopls4 = RNHS2errCAopls4/np.sqrt(40)


# Pointers for CA and CH3
CH3pointOpls4 = np.array([],dtype=int)
CApointOpls4 = np.array([],dtype=int)

for i, canum in enumerate(RNHnumCH3opls4):
    if canum in RNHnumCAopls4:
        j = np.where(RNHnumCAopls4==canum)
        CH3pointOpls4 = np.append(CH3pointOpls4,i)
        CApointOpls4 = np.append(CApointOpls4,j)
    
RNHidCH3opls4 = RNHidCH3opls4[CH3pointOpls4]
RNHresCH3opls4 = RNHresCH3opls4[CH3pointOpls4]
RNHnumCH3opls4 = RNHnumCH3opls4[CH3pointOpls4]
RNHtypeCH3opls4 = RNHtypeCH3opls4[CH3pointOpls4]
RNHS2CH3opls4 = RNHS2CH3opls4[CH3pointOpls4]
RNHS2errCH3opls4 = RNHS2errCH3opls4[CH3pointOpls4]

RNHidCAopls4=RNHresCAopls4[CApointOpls4]
RNHresCAopls4=RNHresCAopls4[CApointOpls4]
RNHnumCAopls4=RNHnumCAopls4[CApointOpls4]
RNHS2CAopls4=RNHS2CAopls4[CApointOpls4]
RNHS2errCAopls4=RNHS2errCAopls4[CApointOpls4]



In [ ]:
# Read Best fit fractional 2H data

#Format
#Methyl, Model, X2, taum, taum_error, S2, S2_error, Sf2, Sf2_error, Ss2, Ss2_error, tauf, tauf_error, taus, taus_error

ch3file=open('./Experimental_modelfree/EcRNH_CH3_Threefield_MCtm.csv','r')

FracRNHidCH3MF = []
FracRNHresCH3MF = []
FracRNHnumCH3MF = np.array([],dtype=int)
FracRNHtypeCH3MF = []
FracRNHmodCH3MF = np.array([],dtype=int)
FracRNHS2CH3MF = np.array([])
FracRNHS2errCH3MF = np.array([])
FracRNHSf2CH3MF = np.array([])
FracRNHSf2errCH3MF = np.array([])
FracRNHSs2CH3MF = np.array([])
FracRNHSs2errCH3MF = np.array([])
FracRNHtfCH3MF = np.array([])
FracRNHtferrCH3MF = np.array([])
FracRNHtsCH3MF = np.array([])
FracRNHtserrCH3MF = np.array([])
FracRNHfixtmCH3MF = np.array([])

for line in ch3file:
    line0=line.strip()
    
    if len(line0) > 0 and not re.match("Methyl",line0) and not re.match("#" ,line0):
        tmpdata=line0.split(',')
        FracRNHidCH3MF=np.append(FracRNHidCH3MF,tmpdata[0])
        if tmpdata[0][0] == "A":
            FracRNHresCH3MF=np.append(FracRNHresCH3MF,"ALA")
        elif tmpdata[0][0] == "I":
            FracRNHresCH3MF=np.append(FracRNHresCH3MF,"ILE")
        elif tmpdata[0][0] == "L":
            FracRNHresCH3MF=np.append(FracRNHresCH3MF,"LEU")
        elif tmpdata[0][0] == "M":
            FracRNHresCH3MF=np.append(FracRNHresCH3MF,"MET")
        elif tmpdata[0][0] == "T":
            FracRNHresCH3MF=np.append(FracRNHresCH3MF,"THR")
        elif tmpdata[0][0] == "V":
            FracRNHresCH3MF=np.append(FracRNHresCH3MF,"VAL")
    
        FracRNHnumCH3MF=np.append(FracRNHnumCH3MF,float(tmpdata[0][1:tmpdata[0].find("C")]))
        FracRNHtypeCH3MF=np.append(FracRNHtypeCH3MF,tmpdata[0][tmpdata[0].find("C"):])
        FracRNHmodCH3MF=np.append(FracRNHmodCH3MF,int(float(tmpdata[1])))
        FracRNHfixtmCH3MF=np.append(FracRNHfixtmCH3MF,float(tmpdata[3]))
        FracRNHS2CH3MF=np.append(FracRNHS2CH3MF,float(tmpdata[5]))
        FracRNHS2errCH3MF=np.append(FracRNHS2errCH3MF,float(tmpdata[6]))
        FracRNHSf2CH3MF=np.append(FracRNHSf2CH3MF,float(tmpdata[7]))
        FracRNHSf2errCH3MF=np.append(FracRNHSf2errCH3MF,float(tmpdata[8]))
        FracRNHSs2CH3MF=np.append(FracRNHSs2CH3MF,float(tmpdata[9]))
        FracRNHSs2errCH3MF=np.append(FracRNHSs2errCH3MF,float(tmpdata[10]))
        FracRNHtfCH3MF=np.append(FracRNHtfCH3MF,float(tmpdata[11]))
        FracRNHtferrCH3MF=np.append(FracRNHtferrCH3MF,float(tmpdata[12]))
        FracRNHtsCH3MF=np.append(FracRNHtsCH3MF,float(tmpdata[13]))
        FracRNHtserrCH3MF=np.append(FracRNHtserrCH3MF,float(tmpdata[14]))

ch3file.close()

OPLS4FracCH3pointMF = np.array([],dtype=int)
FracOPLS4CH3pointMF = np.array([],dtype=int)

for i, simid in enumerate(RNHidCH3opls4):
    if simid in FracRNHidCH3MF:
        j = np.where(FracRNHidCH3MF==simid)
        OPLS4FracCH3pointMF = np.append(OPLS4FracCH3pointMF,i)
        FracOPLS4CH3pointMF = np.append(FracOPLS4CH3pointMF,j[0][0])
        
#print(np.column_stack((FracRNHidCH3MF[FracOPLS4CH3pointMF],RNHidCH3opls4[OPLS4FracCH3pointMF])))

## Analyze rotamer jumps

In [ ]:
# unblocked order parameters# set block
nsize = np.shape(RNHchiOpls4)[0]

nblock = int(np.ceil(np.shape(RNHchiOpls4)[0]/nsize))

chi1pointer = np.array([],dtype=int)
chi2pointer = np.array([],dtype=int)
LeuCD1pointer = np.array([],dtype=int)
LeuCD2pointer = np.array([],dtype=int)

for i, val in enumerate(RNHchiname):
    if val[0:3] == "LEU":
        if val[-4:] == "CHI1":
            chi1pointer = np.append(chi1pointer,i)
        elif val[-4:] == "CHI2":
            chi2pointer = np.append(chi2pointer,i)
            
nchi = len(chi1pointer)
LeuChiNames = RNHchiname[chi1pointer]

for i, val in enumerate(RNHidCH3opls4):
    if val[0] == "L" and RNHtypeCH3opls4[i] == "CD1":
        tmp = "LEU_"+val[1:val.find("C")]+"_CHI1"
        if tmp in LeuChiNames:
            LeuCD1pointer = np.append(LeuCD1pointer,i)
    if val[0] == "L" and RNHtypeCH3opls4[i] == "CD2":
        tmp = "LEU_"+val[1:val.find("C")]+"_CHI1"
        if tmp in LeuChiNames:
            LeuCD2pointer = np.append(LeuCD2pointer,i)

LeupopChi1 = np.zeros([nchi,3])
LeuS2rotamer1 = np.zeros(nchi)
LeuS2iso1 = np.zeros(nchi)
LeuS2jump1 = np.zeros(nchi)

LeupopChi2 = np.zeros([nchi,3])
LeuS2rotamer2 = np.zeros(nchi)
LeuS2iso2 = np.zeros(nchi)
LeuS2jump2 = np.zeros(nchi)

Leupopchi12 = np.zeros([nchi,9])
LeuS2iso12 = np.zeros(nchi)
LeuS2jump12 = np.zeros(nchi)
LeuS2rotamer12 = np.zeros(nchi)

for i, val in enumerate(chi1pointer):
    print("Methyl",RNHchiname[val])
    
    L1dihed = RNHchiOpls4[:,val]
    L2dihed = RNHchiOpls4[:,chi2pointer[i]]

    for j, val in enumerate(L1dihed):
        if val < 0:
            L1dihed[j] = val + 360  
    for j, val in enumerate(L2dihed):
        if val < 0:
            L2dihed[j] = val + 360
        
    for k in range(0,nblock):
        #print(k,end=" ")
        istart = int(nsize*k)
        iend = int(istart+nsize)
    
        # analyze chi1
        tmp = jumpS2Chi1(L1dihed[istart:iend])
        LeupopChi1[i,:] = LeupopChi1[i,:] + tmp[0]
        LeuS2rotamer1[i] = LeuS2rotamer1[i] + tmp[1]
        LeuS2iso1[i] = LeuS2iso1[i] + tmp[2] 
        LeuS2jump1[i] = LeuS2jump1[i] + tmp[3]

        # analyze chi2
        tmp = jumpS2Chi1(L2dihed[istart:iend])
        LeupopChi2[i,:] = LeupopChi2[i,:] + tmp[0]
        LeuS2rotamer2[i] = LeuS2rotamer2[i] + tmp[1]
        LeuS2iso2[i] = LeuS2iso2[i] + tmp[2] 
        LeuS2jump2[i] = LeuS2jump2[i] + tmp[3]

        # analyze chi1/chi2
        tmp = jumpS2Chi1Chi2(L1dihed[istart:iend],L2dihed[istart:iend])
        Leupopchi12[i,:] = Leupopchi12[i,:] + tmp[0]
        LeuS2rotamer12[i]= LeuS2rotamer12[i] + tmp[1]
        LeuS2iso12[i] = LeuS2iso12[i] + tmp[2]
        LeuS2jump12[i] = LeuS2jump12[i] + tmp[3]

    LeupopChi1[i,:] = LeupopChi1[i,:]/nblock
    LeuS2rotamer1[i] = LeuS2rotamer1[i]/nblock
    LeuS2iso1[i] = LeuS2iso1[i]/nblock
    LeuS2jump1[i] = LeuS2jump1[i]/nblock
    
    LeupopChi2[i,:] = LeupopChi2[i,:]/nblock
    LeuS2rotamer2[i] = LeuS2rotamer2[i]/nblock
    LeuS2iso2[i] = LeuS2iso2[i]/nblock
    LeuS2jump2[i] = LeuS2jump2[i]/nblock
        
    Leupopchi12[i,:] = Leupopchi12[i,:]/nblock
    LeuS2rotamer12[i]= LeuS2rotamer12[i]/nblock
    LeuS2iso12[i] = LeuS2iso12[i]/nblock
    LeuS2jump12[i] = LeuS2jump12[i]/nblock
    
# Ile chi1/chi2 dihedral angles

chi1pointer = np.array([],dtype=int)
chi2pointer = np.array([],dtype=int)
IleCD1pointer = np.array([],dtype=int)
IleCG2pointer = np.array([],dtype=int)

for i, val in enumerate(RNHchiname):
    if val[0:3] == "ILE":
        if val[-4:] == "CHI1":
            chi1pointer = np.append(chi1pointer,i)
        
        elif val[-4:] == "CHI2":
            chi2pointer = np.append(chi2pointer,i)

nchi = len(chi1pointer)
IleChiNames = RNHchiname[chi1pointer]

for i, val in enumerate(RNHidCH3opls4):
    if val[0] == "I" and RNHtypeCH3opls4[i] == "CD1":
        tmp = "ILE_"+val[1:val.find("C")]+"_CHI1"
        if tmp in IleChiNames:
            IleCD1pointer = np.append(IleCD1pointer,i)
    if val[0] == "I" and RNHtypeCH3opls4[i] == "CG2":
        tmp = "ILE_"+val[1:val.find("C")]+"_CHI1"
        if tmp in IleChiNames:
            IleCG2pointer = np.append(IleCG2pointer,i)

IlepopChi1 = np.zeros([nchi,3])
IleS2rotamer1 = np.zeros(nchi)
IleS2iso1 = np.zeros(nchi)
IleS2jump1 = np.zeros(nchi)

IlepopChi2 = np.zeros([nchi,3])
IleS2rotamer2 = np.zeros(nchi)
IleS2iso2 = np.zeros(nchi)
IleS2jump2 = np.zeros(nchi)

Ilepopchi12 = np.zeros([nchi,9])
IleS2iso12 = np.zeros(nchi)
IleS2jump12 = np.zeros(nchi)
IleS2rotamer12 = np.zeros(nchi)

for i, val in enumerate(chi1pointer):
    print("Methyl",RNHchiname[val])
    
    L1dihed = RNHchiOpls4[:,val]
    L2dihed = RNHchiOpls4[:,chi2pointer[i]]

    for j, val in enumerate(L1dihed):
        if val < 0:
            L1dihed[j] = val + 360  
    for j, val in enumerate(L2dihed):
        if val < 0:
            L2dihed[j] = val + 360
        
    for k in range(0,nblock):
        #print(k,end=" ")
        istart = int(nsize*k)
        iend = int(istart+nsize)
    
        # analyze chi1
        tmp = jumpS2Chi1(L1dihed[istart:iend])
        IlepopChi1[i,:] = IlepopChi1[i,:] + tmp[0]
        IleS2rotamer1[i] = IleS2rotamer1[i] + tmp[1]
        IleS2iso1[i] = IleS2iso1[i] + tmp[2] 
        IleS2jump1[i] = IleS2jump1[i] + tmp[3]

        # analyze chi2
        tmp = jumpS2Chi1(L2dihed[istart:iend])
        IlepopChi2[i,:] = IlepopChi2[i,:] + tmp[0]
        IleS2rotamer2[i] = IleS2rotamer2[i] + tmp[1]
        IleS2iso2[i] = IleS2iso2[i] + tmp[2] 
        IleS2jump2[i] = IleS2jump2[i] + tmp[3]

        # analyze chi1/chi2
        tmp = jumpS2Chi1Chi2(L1dihed[istart:iend],L2dihed[istart:iend])
        Ilepopchi12[i,:] = Ilepopchi12[i,:] + tmp[0]
        IleS2rotamer12[i]= IleS2rotamer12[i] + tmp[1]
        IleS2iso12[i] = IleS2iso12[i] + tmp[2]
        IleS2jump12[i] = IleS2jump12[i] + tmp[3]

    IlepopChi1[i,:] = IlepopChi1[i,:]/nblock
    IleS2rotamer1[i] = IleS2rotamer1[i]/nblock
    IleS2iso1[i] = IleS2iso1[i]/nblock
    IleS2jump1[i] = IleS2jump1[i]/nblock
    
    IlepopChi2[i,:] = IlepopChi2[i,:]/nblock
    IleS2rotamer2[i] = IleS2rotamer2[i]/nblock
    IleS2iso2[i] = IleS2iso2[i]/nblock
    IleS2jump2[i] = IleS2jump2[i]/nblock
        
    Ilepopchi12[i,:] = Ilepopchi12[i,:]/nblock
    IleS2rotamer12[i]= IleS2rotamer12[i]/nblock
    IleS2iso12[i] = IleS2iso12[i]/nblock
    IleS2jump12[i] = IleS2jump12[i]/nblock
    
# Thr chi1 dihedral angles

chi1pointer = np.array([],dtype=int)
ThrCG2pointer = np.array([],dtype=int)

for i, val in enumerate(RNHchiname):
    if val[0:3] == "THR":
        if val[-4:] == "CHI1":
            chi1pointer = np.append(chi1pointer,i)

nchi = len(chi1pointer)
ThrChiNames = RNHchiname[chi1pointer]

for i, val in enumerate(RNHidCH3opls4):
    if val[0] == "T" and RNHtypeCH3opls4[i] == "CG2":
        tmp = "THR_"+val[1:val.find("C")]+"_CHI1"
        if tmp in ThrChiNames:
            ThrCG2pointer = np.append(ThrCG2pointer,i)
            
ThrpopChi1 = np.zeros([nchi,3])
ThrS2rotamer1 = np.zeros(nchi)
ThrS2iso1 = np.zeros(nchi)
ThrS2jump1 = np.zeros(nchi)

for i, val in enumerate(chi1pointer):
    print("Methyl",RNHchiname[val])
    
    L1dihed = RNHchiOpls4[:,val]

    for j, val in enumerate(L1dihed):
        if val < 0:
            L1dihed[j] = val + 360  
        
    for k in range(0,nblock):
        #print(k,end=" ")
        istart = int(nsize*k)
        iend = int(istart+nsize)
    
        # analyze chi1
        tmp = jumpS2Chi1(L1dihed[istart:iend])
        ThrpopChi1[i,:] = ThrpopChi1[i,:] + tmp[0]
        ThrS2rotamer1[i] = ThrS2rotamer1[i] + tmp[1]
        ThrS2iso1[i] = ThrS2iso1[i] + tmp[2] 
        ThrS2jump1[i] = ThrS2jump1[i] + tmp[3]

    ThrpopChi1[i,:] = ThrpopChi1[i,:]/nblock
    ThrS2rotamer1[i] = ThrS2rotamer1[i]/nblock
    ThrS2iso1[i] = ThrS2iso1[i]/nblock
    ThrS2jump1[i] = ThrS2jump1[i]/nblock
    
# Val chi1 dihedral angles

chi1pointer = np.array([],dtype=int)
ValCG1pointer = np.array([],dtype=int)
ValCG2pointer = np.array([],dtype=int)

for i, val in enumerate(RNHchiname):
    if val[0:3] == "VAL":
        if val[-4:] == "CHI1":
            chi1pointer = np.append(chi1pointer,i)
            
nchi = len(chi1pointer)
ValChiNames = RNHchiname[chi1pointer]

for i, val in enumerate(RNHidCH3opls4):
    if val[0] == "V" and RNHtypeCH3opls4[i] == "CG1":
        tmp = "VAL_"+val[1:val.find("C")]+"_CHI1"
        if tmp in ValChiNames:
            ValCG1pointer = np.append(ValCG1pointer,i)
    if val[0] == "V" and RNHtypeCH3opls4[i] == "CG2":
        tmp = "VAL_"+val[1:val.find("C")]+"_CHI1"
        if tmp in ValChiNames:
            ValCG2pointer = np.append(ValCG2pointer,i)
            


ValpopChi1 = np.zeros([nchi,3])
ValS2rotamer1 = np.zeros(nchi)
ValS2iso1 = np.zeros(nchi)
ValS2jump1 = np.zeros(nchi)

for i, val in enumerate(chi1pointer):
    print("Methyl",RNHchiname[val])
    
    L1dihed = RNHchiOpls4[:,val]

    for j, val in enumerate(L1dihed):
        if val < 0:
            L1dihed[j] = val + 360  
    
    for k in range(0,nblock):
        #print(k,end=" ")
        istart = int(nsize*k)
        iend = int(istart+nsize)
    
        # analyze chi1
        tmp = jumpS2Chi1(L1dihed[istart:iend])
        ValpopChi1[i,:] = ValpopChi1[i,:] + tmp[0]
        ValS2rotamer1[i] = ValS2rotamer1[i] + tmp[1]
        ValS2iso1[i] = ValS2iso1[i] + tmp[2] 
        ValS2jump1[i] = ValS2jump1[i] + tmp[3]

    ValpopChi1[i,:] = ValpopChi1[i,:]/nblock
    ValS2rotamer1[i] = ValS2rotamer1[i]/nblock
    ValS2iso1[i] = ValS2iso1[i]/nblock
    ValS2jump1[i] = ValS2jump1[i]/nblock
    
    if nblock == 1:
        LeupopChi1nb = LeupopChi1.copy()
        LeuS2rotamer1nb = LeuS2rotamer1.copy()
        LeuS2iso1nb = LeuS2iso1.copy()
        LeuS2jump1nb = LeuS2jump1.copy()
    
        LeupopChi2nb = LeupopChi2.copy()
        LeuS2rotamer2nb = LeuS2rotamer2.copy()
        LeuS2iso2nb = LeuS2iso2.copy()
        LeuS2jump2nb = LeuS2jump2.copy()
        
        Leupopchi12nb = Leupopchi12.copy()
        LeuS2rotamer12nb= LeuS2rotamer12.copy()
        LeuS2iso12nb = LeuS2iso12.copy()
        LeuS2jump12nb = LeuS2jump12.copy()
        
        IlepopChi1nb = IlepopChi1.copy()
        IleS2rotamer1nb = IleS2rotamer1.copy()
        IleS2iso1nb = IleS2iso1.copy()
        IleS2jump1nb = IleS2jump1.copy()
    
        IlepopChi2nb = IlepopChi2.copy()
        IleS2rotamer2nb = IleS2rotamer2.copy()
        IleS2iso2nb = IleS2iso2.copy()
        IleS2jump2nb = IleS2jump2.copy()
        
        Ilepopchi12nb = Ilepopchi12.copy()
        IleS2rotamer12nb = IleS2rotamer12.copy()
        IleS2iso12nb = IleS2iso12.copy()
        IleS2jump12nb = IleS2jump12.copy()
        
        ThrpopChi1nb = ThrpopChi1.copy()
        ThrS2rotamer1nb = ThrS2rotamer1.copy()
        ThrS2iso1nb = ThrS2iso1.copy()
        ThrS2jump1nb = ThrS2jump1.copy()
    
        ValpopChi1nb = ValpopChi1.copy()
        ValS2rotamer1nb = ValS2rotamer1.copy()
        ValS2iso1nb = ValS2iso1.copy()
        ValS2jump1nb = ValS2jump1.copy()
        
    

In [ ]:
# unblocked order parameters

fig = plt.figure(figsize=(8,4))

plt.subplot(121, aspect=1)

Chi12S2rotamer1ave = np.mean(np.append(LeuS2rotamer1nb,IleS2rotamer1nb))
Chi12S2rotamer2ave = np.mean(np.append(LeuS2rotamer2nb,IleS2rotamer2nb))
print("Rotamer 2 angles",Chi12S2rotamer1ave, Chi12S2rotamer2ave,Chi12S2rotamer1ave*Chi12S2rotamer2ave)

Chi1S2rotamer1ave = np.mean(np.concatenate((IleS2rotamer1nb,ThrS2rotamer1nb,ValS2rotamer1nb)))
print("Rotamer 1 angle",Chi1S2rotamer1ave)

plt.plot(RNHS2CH3opls4[IleCG2pointer],Chi1S2rotamer1ave*IleS2jump1nb*RNHS2CAopls4[IleCG2pointer],marker='.',
         c=ReddishPurple,markersize=8,lw=0)

plt.plot(RNHS2CH3opls4[ThrCG2pointer],Chi1S2rotamer1ave*ThrS2jump1nb*RNHS2CAopls4[ThrCG2pointer],marker='.',
         c=Orange,markersize=8,lw=0)

tmp = (RNHS2CH3opls4[ValCG1pointer]+RNHS2CH3opls4[ValCG2pointer])/2
plt.plot(tmp,Chi1S2rotamer1ave*ValS2jump1nb*RNHS2CAopls4[ValCG1pointer],marker='.',
         c=Blue,markersize=8,lw=0)

plt.plot([0,1],[0,1],c='black')
plt.xlabel(r'$S^2_{axis}$ (Conventional)', fontsize=14)
plt.ylabel(r'$S^2_{axis}$ ($\chi_1$ jump)', fontsize=14)

plt.text(0.05,0.9,"(a)",fontsize=12)

plt.xlim(0,1)
plt.ylim(0,1)

plt.subplot(122, aspect=1)

Chi12S2rotamer12ave = np.mean(np.append(LeuS2rotamer12nb,IleS2rotamer12nb))
print("Rotamer 2 angles",Chi12S2rotamer12ave )

tmp = (RNHS2CH3opls4[LeuCD1pointer]+RNHS2CH3opls4[LeuCD2pointer])/2
plt.plot(tmp,Chi12S2rotamer12ave*LeuS2jump12nb*RNHS2CAopls4[LeuCD1pointer],marker='.',
         c='black',markersize=8,lw=0)

print(np.column_stack([RNHidCH3opls4[LeuCD1pointer],RNHS2CH3opls4[LeuCD1pointer],Chi12S2rotamer12ave*LeuS2jump12*RNHS2CAopls4[LeuCD1pointer]]))

plt.plot(RNHS2CH3opls4[IleCD1pointer],
         Chi12S2rotamer12ave*IleS2jump12nb*RNHS2CAopls4[IleCD1pointer],marker='.',
         c=ReddishPurple,markersize=8,lw=0)

plt.plot([0,1],[0,1],c='black')
plt.xlabel(r'$S^2_{axis}$ (Conventional)', fontsize=14)
plt.ylabel(r'$S^2_{axis}$ ($\chi_1/\chi_2$ jump)', fontsize=14)

plt.text(0.05,0.9,"(b)",fontsize=12)

plt.xlim(0,1)
plt.ylim(0,1)

plt.tight_layout()

#plt.savefig("unblocked_methyl_chi_angle_S2.png",dpi=300,pad_inches=0)
#plt.savefig("unblocked_methyl_chi_angle_S2.pdf",pad_inches=0)

plt.show()

In [ ]:
# blocked order parameters
# set block size
nsize = 5000

nblock = int(np.ceil(np.shape(RNHchiOpls4)[0]/nsize))

chi1pointer = np.array([],dtype=int)
chi2pointer = np.array([],dtype=int)
LeuCD1pointer = np.array([],dtype=int)
LeuCD2pointer = np.array([],dtype=int)

for i, val in enumerate(RNHchiname):
    if val[0:3] == "LEU":
        if val[-4:] == "CHI1":
            chi1pointer = np.append(chi1pointer,i)
        elif val[-4:] == "CHI2":
            chi2pointer = np.append(chi2pointer,i)
            
nchi = len(chi1pointer)
LeuChiNames = RNHchiname[chi1pointer]

for i, val in enumerate(RNHidCH3opls4):
    if val[0] == "L" and RNHtypeCH3opls4[i] == "CD1":
        tmp = "LEU_"+val[1:val.find("C")]+"_CHI1"
        if tmp in LeuChiNames:
            LeuCD1pointer = np.append(LeuCD1pointer,i)
    if val[0] == "L" and RNHtypeCH3opls4[i] == "CD2":
        tmp = "LEU_"+val[1:val.find("C")]+"_CHI1"
        if tmp in LeuChiNames:
            LeuCD2pointer = np.append(LeuCD2pointer,i)

LeupopChi1 = np.zeros([nchi,3])
LeuS2rotamer1 = np.zeros(nchi)
LeuS2iso1 = np.zeros(nchi)
LeuS2jump1 = np.zeros(nchi)

LeupopChi2 = np.zeros([nchi,3])
LeuS2rotamer2 = np.zeros(nchi)
LeuS2iso2 = np.zeros(nchi)
LeuS2jump2 = np.zeros(nchi)

Leupopchi12 = np.zeros([nchi,9])
LeuS2iso12 = np.zeros(nchi)
LeuS2jump12 = np.zeros(nchi)
LeuS2rotamer12 = np.zeros(nchi)

for i, val in enumerate(chi1pointer):
    print("Methyl",RNHchiname[val])
    
    L1dihed = RNHchiOpls4[:,val]
    L2dihed = RNHchiOpls4[:,chi2pointer[i]]

    for j, val in enumerate(L1dihed):
        if val < 0:
            L1dihed[j] = val + 360  
    for j, val in enumerate(L2dihed):
        if val < 0:
            L2dihed[j] = val + 360
        
    for k in range(0,nblock):
        #print(k,end=" ")
        istart = int(nsize*k)
        iend = int(istart+nsize)
    
        # analyze chi1
        tmp = jumpS2Chi1(L1dihed[istart:iend])
        LeupopChi1[i,:] = LeupopChi1[i,:] + tmp[0]
        LeuS2rotamer1[i] = LeuS2rotamer1[i] + tmp[1]
        LeuS2iso1[i] = LeuS2iso1[i] + tmp[2] 
        LeuS2jump1[i] = LeuS2jump1[i] + tmp[3]

        # analyze chi2
        tmp = jumpS2Chi1(L2dihed[istart:iend])
        LeupopChi2[i,:] = LeupopChi2[i,:] + tmp[0]
        LeuS2rotamer2[i] = LeuS2rotamer2[i] + tmp[1]
        LeuS2iso2[i] = LeuS2iso2[i] + tmp[2] 
        LeuS2jump2[i] = LeuS2jump2[i] + tmp[3]

        # analyze chi1/chi2
        tmp = jumpS2Chi1Chi2(L1dihed[istart:iend],L2dihed[istart:iend])
        Leupopchi12[i,:] = Leupopchi12[i,:] + tmp[0]
        LeuS2rotamer12[i]= LeuS2rotamer12[i] + tmp[1]
        LeuS2iso12[i] = LeuS2iso12[i] + tmp[2]
        LeuS2jump12[i] = LeuS2jump12[i] + tmp[3]

    LeupopChi1[i,:] = LeupopChi1[i,:]/nblock
    LeuS2rotamer1[i] = LeuS2rotamer1[i]/nblock
    LeuS2iso1[i] = LeuS2iso1[i]/nblock
    LeuS2jump1[i] = LeuS2jump1[i]/nblock
    
    LeupopChi2[i,:] = LeupopChi2[i,:]/nblock
    LeuS2rotamer2[i] = LeuS2rotamer2[i]/nblock
    LeuS2iso2[i] = LeuS2iso2[i]/nblock
    LeuS2jump2[i] = LeuS2jump2[i]/nblock
        
    Leupopchi12[i,:] = Leupopchi12[i,:]/nblock
    LeuS2rotamer12[i]= LeuS2rotamer12[i]/nblock
    LeuS2iso12[i] = LeuS2iso12[i]/nblock
    LeuS2jump12[i] = LeuS2jump12[i]/nblock
    
# Ile chi1/chi2 dihedral angles

chi1pointer = np.array([],dtype=int)
chi2pointer = np.array([],dtype=int)
IleCD1pointer = np.array([],dtype=int)
IleCG2pointer = np.array([],dtype=int)

for i, val in enumerate(RNHchiname):
    if val[0:3] == "ILE":
        if val[-4:] == "CHI1":
            chi1pointer = np.append(chi1pointer,i)
        
        elif val[-4:] == "CHI2":
            chi2pointer = np.append(chi2pointer,i)

nchi = len(chi1pointer)
IleChiNames = RNHchiname[chi1pointer]

for i, val in enumerate(RNHidCH3opls4):
    if val[0] == "I" and RNHtypeCH3opls4[i] == "CD1":
        tmp = "ILE_"+val[1:val.find("C")]+"_CHI1"
        if tmp in IleChiNames:
            IleCD1pointer = np.append(IleCD1pointer,i)
    if val[0] == "I" and RNHtypeCH3opls4[i] == "CG2":
        tmp = "ILE_"+val[1:val.find("C")]+"_CHI1"
        if tmp in IleChiNames:
            IleCG2pointer = np.append(IleCG2pointer,i)

IlepopChi1 = np.zeros([nchi,3])
IleS2rotamer1 = np.zeros(nchi)
IleS2iso1 = np.zeros(nchi)
IleS2jump1 = np.zeros(nchi)

IlepopChi2 = np.zeros([nchi,3])
IleS2rotamer2 = np.zeros(nchi)
IleS2iso2 = np.zeros(nchi)
IleS2jump2 = np.zeros(nchi)

Ilepopchi12 = np.zeros([nchi,9])
IleS2iso12 = np.zeros(nchi)
IleS2jump12 = np.zeros(nchi)
IleS2rotamer12 = np.zeros(nchi)

for i, val in enumerate(chi1pointer):
    print("Methyl",RNHchiname[val])
    
    L1dihed = RNHchiOpls4[:,val]
    L2dihed = RNHchiOpls4[:,chi2pointer[i]]

    for j, val in enumerate(L1dihed):
        if val < 0:
            L1dihed[j] = val + 360  
    for j, val in enumerate(L2dihed):
        if val < 0:
            L2dihed[j] = val + 360
        
    for k in range(0,nblock):
        #print(k,end=" ")
        istart = int(nsize*k)
        iend = int(istart+nsize)
    
        # analyze chi1
        tmp = jumpS2Chi1(L1dihed[istart:iend])
        IlepopChi1[i,:] = IlepopChi1[i,:] + tmp[0]
        IleS2rotamer1[i] = IleS2rotamer1[i] + tmp[1]
        IleS2iso1[i] = IleS2iso1[i] + tmp[2] 
        IleS2jump1[i] = IleS2jump1[i] + tmp[3]

        # analyze chi2
        tmp = jumpS2Chi1(L2dihed[istart:iend])
        IlepopChi2[i,:] = IlepopChi2[i,:] + tmp[0]
        IleS2rotamer2[i] = IleS2rotamer2[i] + tmp[1]
        IleS2iso2[i] = IleS2iso2[i] + tmp[2] 
        IleS2jump2[i] = IleS2jump2[i] + tmp[3]

        # analyze chi1/chi2
        tmp = jumpS2Chi1Chi2(L1dihed[istart:iend],L2dihed[istart:iend])
        Ilepopchi12[i,:] = Ilepopchi12[i,:] + tmp[0]
        IleS2rotamer12[i]= IleS2rotamer12[i] + tmp[1]
        IleS2iso12[i] = IleS2iso12[i] + tmp[2]
        IleS2jump12[i] = IleS2jump12[i] + tmp[3]

    IlepopChi1[i,:] = IlepopChi1[i,:]/nblock
    IleS2rotamer1[i] = IleS2rotamer1[i]/nblock
    IleS2iso1[i] = IleS2iso1[i]/nblock
    IleS2jump1[i] = IleS2jump1[i]/nblock
    
    IlepopChi2[i,:] = IlepopChi2[i,:]/nblock
    IleS2rotamer2[i] = IleS2rotamer2[i]/nblock
    IleS2iso2[i] = IleS2iso2[i]/nblock
    IleS2jump2[i] = IleS2jump2[i]/nblock
        
    Ilepopchi12[i,:] = Ilepopchi12[i,:]/nblock
    IleS2rotamer12[i]= IleS2rotamer12[i]/nblock
    IleS2iso12[i] = IleS2iso12[i]/nblock
    IleS2jump12[i] = IleS2jump12[i]/nblock
    
# Thr chi1 dihedral angles

chi1pointer = np.array([],dtype=int)
ThrCG2pointer = np.array([],dtype=int)

for i, val in enumerate(RNHchiname):
    if val[0:3] == "THR":
        if val[-4:] == "CHI1":
            chi1pointer = np.append(chi1pointer,i)

nchi = len(chi1pointer)
ThrChiNames = RNHchiname[chi1pointer]

for i, val in enumerate(RNHidCH3opls4):
    if val[0] == "T" and RNHtypeCH3opls4[i] == "CG2":
        tmp = "THR_"+val[1:val.find("C")]+"_CHI1"
        if tmp in ThrChiNames:
            ThrCG2pointer = np.append(ThrCG2pointer,i)
            
ThrpopChi1 = np.zeros([nchi,3])
ThrS2rotamer1 = np.zeros(nchi)
ThrS2iso1 = np.zeros(nchi)
ThrS2jump1 = np.zeros(nchi)

for i, val in enumerate(chi1pointer):
    print("Methyl",RNHchiname[val])
    
    L1dihed = RNHchiOpls4[:,val]

    for j, val in enumerate(L1dihed):
        if val < 0:
            L1dihed[j] = val + 360  
        
    for k in range(0,nblock):
        #print(k,end=" ")
        istart = int(nsize*k)
        iend = int(istart+nsize)
    
        # analyze chi1
        tmp = jumpS2Chi1(L1dihed[istart:iend])
        ThrpopChi1[i,:] = ThrpopChi1[i,:] + tmp[0]
        ThrS2rotamer1[i] = ThrS2rotamer1[i] + tmp[1]
        ThrS2iso1[i] = ThrS2iso1[i] + tmp[2] 
        ThrS2jump1[i] = ThrS2jump1[i] + tmp[3]

    ThrpopChi1[i,:] = ThrpopChi1[i,:]/nblock
    ThrS2rotamer1[i] = ThrS2rotamer1[i]/nblock
    ThrS2iso1[i] = ThrS2iso1[i]/nblock
    ThrS2jump1[i] = ThrS2jump1[i]/nblock
    
# Val chi1 dihedral angles

chi1pointer = np.array([],dtype=int)
ValCG1pointer = np.array([],dtype=int)
ValCG2pointer = np.array([],dtype=int)

for i, val in enumerate(RNHchiname):
    if val[0:3] == "VAL":
        if val[-4:] == "CHI1":
            chi1pointer = np.append(chi1pointer,i)
            
nchi = len(chi1pointer)
ValChiNames = RNHchiname[chi1pointer]

for i, val in enumerate(RNHidCH3opls4):
    if val[0] == "V" and RNHtypeCH3opls4[i] == "CG1":
        tmp = "VAL_"+val[1:val.find("C")]+"_CHI1"
        if tmp in ValChiNames:
            ValCG1pointer = np.append(ValCG1pointer,i)
    if val[0] == "V" and RNHtypeCH3opls4[i] == "CG2":
        tmp = "VAL_"+val[1:val.find("C")]+"_CHI1"
        if tmp in ValChiNames:
            ValCG2pointer = np.append(ValCG2pointer,i)
            


ValpopChi1 = np.zeros([nchi,3])
ValS2rotamer1 = np.zeros(nchi)
ValS2iso1 = np.zeros(nchi)
ValS2jump1 = np.zeros(nchi)

for i, val in enumerate(chi1pointer):
    print("Methyl",RNHchiname[val])
    
    L1dihed = RNHchiOpls4[:,val]

    for j, val in enumerate(L1dihed):
        if val < 0:
            L1dihed[j] = val + 360  
    
    for k in range(0,nblock):
        #print(k,end=" ")
        istart = int(nsize*k)
        iend = int(istart+nsize)
    
        # analyze chi1
        tmp = jumpS2Chi1(L1dihed[istart:iend])
        ValpopChi1[i,:] = ValpopChi1[i,:] + tmp[0]
        ValS2rotamer1[i] = ValS2rotamer1[i] + tmp[1]
        ValS2iso1[i] = ValS2iso1[i] + tmp[2] 
        ValS2jump1[i] = ValS2jump1[i] + tmp[3]

    ValpopChi1[i,:] = ValpopChi1[i,:]/nblock
    ValS2rotamer1[i] = ValS2rotamer1[i]/nblock
    ValS2iso1[i] = ValS2iso1[i]/nblock
    ValS2jump1[i] = ValS2jump1[i]/nblock
    
    if nblock == 1:
        LeupopChi1nb = LeupopChi1.copy()
        LeuS2rotamer1nb = LeuS2rotamer1.copy()
        LeuS2iso1nb = LeuS2iso1.copy()
        LeuS2jump1nb = LeuS2jump1.copy()
    
        LeupopChi2nb = LeupopChi2.copy()
        LeuS2rotamer2nb = LeuS2rotamer2.copy()
        LeuS2iso2nb = LeuS2iso2.copy()
        LeuS2jump2nb = LeuS2jump2.copy()
        
        Leupopchi12nb = Leupopchi12.copy()
        LeuS2rotamer12nb= LeuS2rotamer12.copy()
        LeuS2iso12nb = LeuS2iso12.copy()
        LeuS2jump12nb = LeuS2jump12.copy()
        
        IlepopChi1nb = IlepopChi1.copy()
        IleS2rotamer1nb = IleS2rotamer1.copy()
        IleS2iso1nb = IleS2iso1.copy()
        IleS2jump1nb = IleS2jump1.copy()
    
        IlepopChi2nb = IlepopChi2.copy()
        IleS2rotamer2nb = IleS2rotamer2.copy()
        IleS2iso2nb = IleS2iso2.copy()
        IleS2jump2nb = IleS2jump2.copy()
        
        Ilepopchi12nb = Ilepopchi12.copy()
        IleS2rotamer12nb = IleS2rotamer12.copy()
        IleS2iso12nb = IleS2iso12.copy()
        IleS2jump12nb = IleS2jump12.copy()
        
        ThrpopChi1nb = ThrpopChi1.copy()
        ThrS2rotamer1nb = ThrS2rotamer1.copy()
        ThrS2iso1nb = ThrS2iso1.copy()
        ThrS2jump1nb = ThrS2jump1.copy()
    
        ValpopChi1nb = ValpopChi1.copy()
        ValS2rotamer1nb = ValS2rotamer1.copy()
        ValS2iso1nb = ValS2iso1.copy()
        ValS2jump1nb = ValS2jump1.copy()
        
    

In [ ]:
# blocked order parameters

fig = plt.figure(figsize=(7,6.5))

plt.subplot(221, aspect=1)

Chi12S2rotamer1ave = np.mean(np.append(LeuS2rotamer1,IleS2rotamer1))
Chi12S2rotamer2ave = np.mean(np.append(LeuS2rotamer2,IleS2rotamer2))
print("Rotamer 2 angles",Chi12S2rotamer1ave, Chi12S2rotamer2ave,Chi12S2rotamer1ave*Chi12S2rotamer2ave)

Chi1S2rotamer1ave = np.mean(np.concatenate((IleS2rotamer1,ThrS2rotamer1,ValS2rotamer1)))
print("Rotamer 1 angle",Chi1S2rotamer1ave)

plt.plot(RNHS2CH3opls4[IleCG2pointer],Chi1S2rotamer1ave*IleS2jump1*RNHS2CAopls4[IleCG2pointer],marker='.',
         c=ReddishPurple,markersize=8,lw=0)

plt.plot(RNHS2CH3opls4[ThrCG2pointer],Chi1S2rotamer1ave*ThrS2jump1*RNHS2CAopls4[ThrCG2pointer],marker='.',
         c=Orange,markersize=8,lw=0)

tmp = (RNHS2CH3opls4[ValCG1pointer]+RNHS2CH3opls4[ValCG2pointer])/2
plt.plot(tmp,Chi1S2rotamer1ave*ValS2jump1*RNHS2CAopls4[ValCG1pointer],marker='.',
         c=Blue,markersize=8,lw=0)

plt.plot([0,1],[0,1],c='black')
#plt.xlabel(r'$S^2_{axis}$ (Conventional)', fontsize=14)
plt.ylabel(r'$S^2_{axis}$ ($\chi_1$ jump)', fontsize=14)

plt.text(0.05,0.9,"(a)",fontsize=12)

plt.xlim(0,1)
plt.ylim(0,1)

plt.subplot(222, aspect=1)

Chi12S2rotamer12ave = np.mean(np.append(LeuS2rotamer12,IleS2rotamer12))
print("Rotamer 2 angles",Chi12S2rotamer12ave )

tmp = (RNHS2CH3opls4[LeuCD1pointer]+RNHS2CH3opls4[LeuCD2pointer])/2
plt.plot(tmp,Chi12S2rotamer12ave*LeuS2jump12*RNHS2CAopls4[LeuCD1pointer],marker='.',
         c='black',markersize=8,lw=0)

plt.plot(RNHS2CH3opls4[IleCD1pointer],
         Chi12S2rotamer12ave*IleS2jump12*RNHS2CAopls4[IleCD1pointer],marker='.',
         c=ReddishPurple,markersize=8,lw=0)

plt.plot([0,1],[0,1],c='black')
#plt.xlabel(r'$S^2_{axis}$ (Conventional)', fontsize=14)
plt.ylabel(r'$S^2_{axis}$ ($\chi_1/\chi_2$ jump)', fontsize=14)

plt.text(0.05,0.9,"(b)",fontsize=12)

plt.xlim(0,1)
plt.ylim(0,1)

plt.subplot(223, aspect=1)

Chi12S2rotamer1ave = np.mean(np.append(LeuS2rotamer1,IleS2rotamer1))
Chi12S2rotamer2ave = np.mean(np.append(LeuS2rotamer2,IleS2rotamer2))
print("Rotamer 2 angles",Chi12S2rotamer1ave, Chi12S2rotamer2ave,Chi12S2rotamer1ave*Chi12S2rotamer2ave)

Chi1S2rotamer1ave = np.mean(np.concatenate((IleS2rotamer1,ThrS2rotamer1,ValS2rotamer1)))
print("Rotamer 1 angle",Chi1S2rotamer1ave)

plt.plot(RNHS2CH3opls4[IleCG2pointer],Chi1S2rotamer1ave*IleS2jump1nb*RNHS2CAopls4[IleCG2pointer],marker='.',
         c=ReddishPurple,markersize=8,lw=0)

plt.plot(RNHS2CH3opls4[ThrCG2pointer],Chi1S2rotamer1ave*ThrS2jump1nb*RNHS2CAopls4[ThrCG2pointer],marker='.',
         c=Orange,markersize=8,lw=0)

tmp = (RNHS2CH3opls4[ValCG1pointer]+RNHS2CH3opls4[ValCG2pointer])/2
plt.plot(tmp,Chi1S2rotamer1ave*ValS2jump1nb*RNHS2CAopls4[ValCG1pointer],marker='.',
         c=Blue,markersize=8,lw=0)

plt.plot([0,1],[0,1],c='black')
plt.xlabel(r'$S^2_{axis}$ (Conventional)', fontsize=14)
plt.ylabel(r'$S^2_{axis}$ ($\chi_1$ jump)', fontsize=14)

plt.text(0.05,0.9,"(c)",fontsize=12)

plt.xlim(0,1)
plt.ylim(0,1)

plt.subplot(224, aspect=1)

Chi12S2rotamer12ave = np.mean(np.append(LeuS2rotamer12nb,IleS2rotamer12))
print("Rotamer 2 angles",Chi12S2rotamer12ave )

tmp = (RNHS2CH3opls4[LeuCD1pointer]+RNHS2CH3opls4[LeuCD2pointer])/2
plt.plot(tmp,Chi12S2rotamer12ave*LeuS2jump12nb*RNHS2CAopls4[LeuCD1pointer],marker='.',
         c='black',markersize=8,lw=0)

plt.plot(RNHS2CH3opls4[IleCD1pointer],
         Chi12S2rotamer12ave*IleS2jump12nb*RNHS2CAopls4[IleCD1pointer],marker='.',
         c=ReddishPurple,markersize=8,lw=0)

plt.plot([0,1],[0,1],c='black')
plt.xlabel(r'$S^2_{axis}$ (Conventional)', fontsize=14)
plt.ylabel(r'$S^2_{axis}$ ($\chi_1/\chi_2$ jump)', fontsize=14)

plt.text(0.05,0.9,"(d)",fontsize=12)

plt.xlim(0,1)
plt.ylim(0,1)

plt.tight_layout()

#plt.savefig("Figure5_methyl_chi_angle_S2.png",dpi=300,pad_inches=0)
#plt.savefig("Figure5_methyl_chi_angle_S2.eps",pad_inches=0)

plt.show()

In [ ]:
# blocked order parameters all on one plot

fig = plt.figure(figsize=(4,4))

plt.subplot(111, aspect=1)

plt.plot(RNHS2CH3opls4[IleCG2pointer],Chi1S2rotamer1ave*IleS2jump1*RNHS2CAopls4[IleCG2pointer],marker='.',
         c=ReddishPurple,markersize=8,lw=0)

plt.plot(RNHS2CH3opls4[ThrCG2pointer],Chi1S2rotamer1ave*ThrS2jump1*RNHS2CAopls4[ThrCG2pointer],marker='.',
         c=Orange,markersize=8,lw=0)

tmp = (RNHS2CH3opls4[ValCG1pointer]+RNHS2CH3opls4[ValCG2pointer])/2
plt.plot(tmp,Chi1S2rotamer1ave*ValS2jump1*RNHS2CAopls4[ValCG1pointer],marker='.',
         c=Blue,markersize=8,lw=0)

plt.xlabel(r'$S^2_{axis}$ (Conventional)', fontsize=14)
plt.ylabel(r'$S^2_{axis}$ (Dihedral Jump)', fontsize=14)

tmp = (RNHS2CH3opls4[LeuCD1pointer]+RNHS2CH3opls4[LeuCD2pointer])/2
plt.plot(tmp,Chi12S2rotamer12ave*LeuS2jump12*RNHS2CAopls4[LeuCD1pointer],marker='s',
         c='black',markersize=4,lw=0)

plt.plot(RNHS2CH3opls4[IleCD1pointer],
         Chi12S2rotamer12ave*IleS2jump12*RNHS2CAopls4[IleCD1pointer],marker='s',
         c=Green,markersize=4,lw=0)

plt.plot([0,1],[0,1],c='black')

plt.xlim(0,1)
plt.ylim(0,1)

plt.tight_layout()

#plt.savefig("All_methyl_chi_angle_S2.png",dpi=300,pad_inches=0)
#plt.savefig("All_methyl_chi_angle_S2.eps",pad_inches=0)

plt.show()

In [ ]:
valJpointer = np.array([1,3,4,5,6,7,8],dtype=int)

valOPLS4 = (RNHS2CH3opls4[ValCG1pointer] +RNHS2CH3opls4[ValCG2pointer])/2

valS2exp = np.array([[54,0.823735167, 0.003862209],
                     [74,0.942529095, 0.0063884],
                     [98,0.663178112, 0.012060945],
                     [101,0.802091059,0.00367521],
                     [121,0.477706294,0.00257345],
                     [153,0.037877829,0.00636922],
                     [155,0.007687609,0.001454917]])

valJcouple = np.array([
    [54,0.33,0.67,0],
    [74,0.04,0.96,0],
    [98,0.44,0.56,0],
    [101,0.85,0.09,0.06],
    [121,0.57,0.43,0],
    [153,0.41,0.48,0.11],
    [155,0.57,0.35,0.08]])

valChemshift = np.array([
    [54,0.43,0.57,0],
    [74,0,1,0],
    [98,0.49,0.0,0.51],
    [101,0.0,0.12,0.88],
    [121,0.33,0.67,0],
    [153,0.15,0.72,0.13],
    [155,0.33,0.65,0.02]])
    
valMerge = valJcouple.copy()
for i in range(0,np.shape(valJcouple)[0]):
    tmpJ = np.sqrt(np.mean((ValpopChi1nb[valJpointer][i,:]-valJcouple[i,1:])**2))
    tmpCS = np.sqrt(np.mean((ValpopChi1nb[valJpointer][i,:]-valChemshift[i,1:])**2))
    print(valChemshift[i,0],tmpJ,tmpCS)
    if tmpCS < tmpJ:
        valMerge[i,:]=valChemshift[i,:]
    
print(valMerge)

valjump = np.zeros(np.shape(valJcouple)[0])
valjumperr = np.zeros(np.shape(valJcouple)[0])

for i in range(0,len(valjump)):
    #valjump[i] = (jumpS2rotamer(valJcouple[i,1:]) + jumpS2rotamer(valChemshift[i,1:]))/2
    valjump[i] = jumpS2rotamer(valMerge[i,1:])
    valjumperr[i] = abs(jumpS2rotamer(valJcouple[i,1:]) - jumpS2rotamer(valChemshift[i,1:]))/np.sqrt(2)

fig=plt.figure(figsize=(3.3,6.5))

plt.subplot(211,aspect=1)

tmp = Chi1S2rotamer1ave*ValS2jump1nb*RNHS2CAopls4[ValCG1pointer] 
plt.errorbar(valS2exp[:,1],tmp[valJpointer],fmt='.',markersize=8,c='black')
tmp = Chi1S2rotamer1ave*ValS2jump1*RNHS2CAopls4[ValCG1pointer] 
plt.errorbar(valS2exp[:,1],tmp[valJpointer],fmt='.',markersize=8,c=ReddishPurple)
plt.plot([0,1],[0,1],c='black')
plt.xlabel(r'$S^2_{axis}$ (Relaxation)', fontsize=12)
plt.ylabel(r'$S^2_{axis}$ ($\chi_1$ jump)', fontsize=12)
plt.text(0.05,0.9,"(a)",fontsize=12)
plt.xlim(0,1)
plt.ylim(0,1)

print(np.sqrt(np.mean((valS2exp[:,1]-valOPLS4[valJpointer])**2)))

plt.subplot(212,aspect=1)
tmpx = Chi1S2rotamer1ave*valjump*(RNHS2CAopls4[ValCG1pointer][valJpointer])
tmpxerr = Chi1S2rotamer1ave*valjumperr*(RNHS2CAopls4[ValCG1pointer][valJpointer])
tmpy = Chi1S2rotamer1ave*ValS2jump1nb*RNHS2CAopls4[ValCG1pointer]
plt.errorbar(tmpx,tmpy[valJpointer], fmt='.',markersize=8,c='black')
plt.plot([0,1],[0,1],c='black')
plt.xlabel(r'$S^2_{axis}$ (J/chemical shift)', fontsize=12)
plt.ylabel(r'$S^2_{axis}$ ($\chi_1$ jump)', fontsize=12)
plt.text(0.05,0.9,"(b)",fontsize=12)
plt.xlim(0,1)
plt.ylim(0,1)

print(np.sqrt(np.mean((tmpx-tmpy[valJpointer])**2)))

plt.tight_layout()

#plt.savefig("Figure6_Val_chi_angle_S2.png",dpi=300,pad_inches=0)
#plt.savefig("Figure6_Val_chi_angle_S2.eps",pad_inches=0)

plt.show()

## Leucine dihedral angle graphs

In [ ]:
Leuchi1pointer = np.array([],dtype=int)
Leuchi2pointer = np.array([],dtype=int)

for i, val in enumerate(RNHchiname):
    if val[0:3] == "LEU": 
        if val[-4:] == "CHI1":
            Leuchi1pointer = np.append(Leuchi1pointer,i)
        elif val[-4:] == "CHI2":
            Leuchi2pointer = np.append(Leuchi2pointer,i)

nchi = len(Leuchi1pointer)

LeuTraj = np.zeros([nchi,np.shape(RNHchiOpls4)[0]])

LeuChiNames = RNHchiname[Leuchi1pointer]

Leutransprob = np.zeros([nchi,9,9])
Leuequilprob = np.zeros([nchi,9])
Leumeanloc = np.zeros([nchi,9,2])

cut=50
npoints = np.shape(RNHchiOpls4)[0]/cut
tdelta = 2e-6/npoints*1e9
print('N samples',int(npoints),'Delta t',tdelta,'ns')

fig = plt.figure(figsize=(6,nchi*2))

for i,val in enumerate(LeuChiNames):  
    L1dihed = RNHchiOpls4[:,Leuchi1pointer[i]]
    L2dihed = RNHchiOpls4[:,Leuchi2pointer[i]]

    j = i+1
    plt.subplot(nchi,1,j)
    
    Leuequilprob[i,:], Leumeanloc[i,:,:],LeuTraj[i,:] = getChi1Chi2(L1dihed,L2dihed)
    tmpstate = LeuTraj[i,0::cut]+4
    
    state0 = tmpstate[0]
    ntransitions = 0
    for k in range(1,len(tmpstate)):
        state1 = tmpstate[k]
        if state1 != state0:
            Leutransprob[i, int(state0),int(state1)] = Leutransprob[i, int(state0),int(state1)] + 1
            ntransitions += 1
            state0 = state1
    Leutransprob[i,:,:] = Leutransprob[i,:,:]/ntransitions/0.01 
    
    plt.plot(LeuTraj[i,0:50000:10],c='black')
    plt.title(val[0:-5])
    plt.ylim(-4.5,4.5)

    for i,val in enumerate([-4,-3,-2,-1,0,1,2,3,4]):
        plt.axhline(val,c=Blue,ls="--",lw=1)

plt.xlabel(r'$t$ (ns)',fontsize=14)

plt.tight_layout()
plt.show()

In [ ]:
nchi = len(LeuChiNames)
nrow = int(np.floor((nchi+1)/2))

fig = plt.figure(figsize=(7,nrow*4))

for i,valname in enumerate(LeuChiNames): 
    plt.subplot(nrow,2,i+1,aspect=1)

    L1dihed = RNHchiOpls4[:,Leuchi1pointer[i]].copy()
    L2dihed = RNHchiOpls4[:,Leuchi2pointer[i]].copy()

    for j,val in enumerate(L1dihed):
        if val < 0:
            L1dihed[j] = val + 360
        if L2dihed[j] < 0:
            L2dihed[j] = L2dihed[j] + 360

    nmax = len(L1dihed)
    
    plt.axhline(120,color=Blue,ls="--",lw=1)
    plt.axhline(240,color=Blue,ls="--",lw=1)
    plt.axvline(120,color=Blue,ls="--",lw=1)
    plt.axvline(240,color=Blue,ls="--",lw=1)

    tmpprobs = (Leutransprob[i,:,:] + np.transpose(Leutransprob[i,:,:]))/2.0

    for j in range(0,8):
        for k in range(j,9):
            if tmpprobs[j,k] > 0:
                tmplw = 3*np.log10(np.maximum(1,100*tmpprobs/np.max(tmpprobs)))
                tmp = np.vstack((Leumeanloc[i,j,:],Leumeanloc[i,k,:]))
                plt.plot(tmp[:,0],tmp[:,1],lw=tmplw[j,k],c=ReddishPurple)
            
    plt.plot(L1dihed[0:nmax:cut],L2dihed[0:nmax:cut],marker='.',markersize=2,lw=0,c='black')

    plt.xlim(0,360)
    plt.ylim(0,360)

    plt.xlabel(r'$\chi_1$ ($^{\circ}$)',fontsize=14)
    plt.ylabel(r'$\chi_2$ ($^{\circ}$)',fontsize=14)
    plt.text(25,325,valname[0:-5],fontsize=12)

plt.tight_layout()

plt.show()

In [ ]:
# Cross-correlation functions
nchi = len(LeuChiNames)
npoints = len(RNHchiOpls4[:,Leuchi1pointer[0]])
tdelta = 2e-6/npoints*1e9
print('N samples',npoints,'Delta t',tdelta,'ns')

LeuChiNames = RNHchiname[Leuchi1pointer]

fig = plt.figure(figsize=(8,nchi*3))

print(np.cos(np.pi))
for i,val in enumerate(LeuChiNames):  
    L1dihed = RNHchiOpls4[:,Leuchi1pointer[i]]
    L2dihed = RNHchiOpls4[:,Leuchi2pointer[i]]
    
    for j,valt in enumerate(L1dihed):
        if valt < 0:
            L1dihed[j] = valt + 360
        if L2dihed[j] < 0:
            L2dihed[j] = L2dihed[j] + 360
            
    corrdim = 5000
    tdim = np.linspace(0,(corrdim-1)*tdelta,corrdim)

    j = 2*i+1
    plt.subplot(nchi,2,j)
    
    acf1 = DihedralCrossCorr(L1dihed*np.pi/180,L2dihed*np.pi/180,corrdim)
    acf2 = DihedralCrossCorr(L2dihed*np.pi/180,L1dihed*np.pi/180,corrdim)
    #acf = (acf1+acf2)/2
    acf = acf1
    
    plt.plot(tdim,acf,c='black')
    plt.title(val[0:-5])
    plt.xlim(0,tdim[-1])
    #plt.ylim(-0.1,1)
    #plt.axhline(0,c=Blue)
    plt.xlabel(r'$\tau$ (ns)',fontsize=14)
    plt.ylabel(r'$C(\tau)$',fontsize=14)

    plt.subplot(nchi,2,j+1)
    
    plt.plot(tdim,acf,c='black')
    plt.title(val[0:-5])
    plt.xlim(0,10)
    #plt.ylim(-0.1,1)
    #plt.axhline(0,c=Blue)
    plt.xlabel(r'$\tau$ (ns)',fontsize=14)

plt.tight_layout()
plt.show()

In [ ]:
statelabels = np.array(["mm","mt","mp","tm","tt","tp","pm","pt","pp"])

statepointer = np.array([[300,300],[300,180],[300,60],
                      [180,300],[180,180],[180,60],
                       [60,300],[60,180],[60,60]])

cut=50
npoints = np.shape(RNHchiOpls4)[0]/cut
tdelta = 2e-6/npoints*1e9
print('N samples',int(npoints),'Delta t',tdelta,'ns')

for ind, nameval in enumerate(LeuChiNames):
    
    print('\n',nameval)

    print("S2",Chi12S2rotamer12ave*LeuS2jump12[ind]*RNHS2CAopls4[LeuCD1pointer[ind]],
          "S2CA",RNHS2CAopls4[LeuCD1pointer[ind]],
          "S2Jump",LeuS2jump12[ind],"\n")
      
    for i in range(0,9):
        print(statelabels[i],np.round(Leupopchi12[ind,i],3),end=" ")

    print("\n")

    for i in range(0,9):
        for j in range(0,9):
            if Leutransprob[ind,i,j] > 1:
                print(statelabels[i],statelabels[j],Leutransprob[ind,i,j])


## Isoleucine dihedral angle plots

In [ ]:
Ilechi1pointer = np.array([],dtype=int)
Ilechi2pointer = np.array([],dtype=int)

for i, val in enumerate(RNHchiname):
    if val[0:3] == "ILE": 
        if val[-4:] == "CHI1":
            Ilechi1pointer = np.append(Ilechi1pointer,i)
        elif val[-4:] == "CHI2":
            Ilechi2pointer = np.append(Ilechi2pointer,i)

nchi = len(Ilechi1pointer)

IleTraj = np.zeros([nchi,np.shape(RNHchiOpls4)[0]])

IleChiNames = RNHchiname[Ilechi1pointer]

cut=50
npoints = np.shape(RNHchiOpls4)[0]/cut
tdelta = 2e-6/npoints*1e9
print('N samples',int(npoints),'Delta t',tdelta,'ns')

Iletransprob = np.zeros([nchi,9,9])
Ileequilprob = np.zeros([nchi,9])
Ilemeanloc = np.zeros([nchi,9,2])

fig = plt.figure(figsize=(6,nchi*2))

for i,val in enumerate(IleChiNames):  
    L1dihed = RNHchiOpls4[:,Ilechi1pointer[i]]
    L2dihed = RNHchiOpls4[:,Ilechi2pointer[i]]

    j = i+1
    plt.subplot(nchi,1,j)
    
    Ileequilprob[i,:], Ilemeanloc[i,:,:],IleTraj[i,:] = getChi1Chi2(L1dihed,L2dihed)
    tmpstate = IleTraj[i,0::cut]+4
    
    state0 = tmpstate[0]
    ntransitions = 0
    for k in range(1,len(tmpstate)):
        state1 = tmpstate[k]
        if state1 != state0:
            Iletransprob[i, int(state0),int(state1)] = Iletransprob[i, int(state0),int(state1)] + 1
            ntransitions += 1
            state0 = state1
    Iletransprob[i,:,:] = Iletransprob[i,:,:]/ntransitions/0.01 
    
    plt.plot(IleTraj[i,0:5000:2],c='black')
    plt.title(val[0:-5])
    plt.ylim(-4.5,4.5)

    for i,val in enumerate([-4,-3,-2,-1,0,1,2,3,4]):
        plt.axhline(val,c=Blue,ls="--",lw=1)

plt.xlabel(r'$t$ (ns)',fontsize=14)

plt.tight_layout()
plt.show()
      

In [ ]:
nchi = len(IleChiNames)
nrow = int(np.floor((nchi+1)/2))

fig = plt.figure(figsize=(7,nrow*4))

for i,valname in enumerate(IleChiNames): 
    plt.subplot(nrow,2,i+1,aspect=1)

    L1dihed = RNHchiOpls4[:,Ilechi1pointer[i]].copy()
    L2dihed = RNHchiOpls4[:,Ilechi2pointer[i]].copy()

    for j,val in enumerate(L1dihed):
        if val < 0:
            L1dihed[j] = val + 360
        if L2dihed[j] < 0:
            L2dihed[j] = L2dihed[j] + 360

    nmax = len(L1dihed)
    cut = 50
    
    plt.axhline(120,color=Blue,ls="--",lw=1)
    plt.axhline(240,color=Blue,ls="--",lw=1)
    plt.axvline(120,color=Blue,ls="--",lw=1)
    plt.axvline(240,color=Blue,ls="--",lw=1)

    tmpprobs = (Iletransprob[i,:,:] + np.transpose(Iletransprob[i,:,:]))/2.0

    for j in range(0,8):
        for k in range(j,9):
            if tmpprobs[j,k] > 0:
                tmplw = 3*np.log10(np.maximum(1,100*tmpprobs/np.max(tmpprobs)))
                tmp = np.vstack((Ilemeanloc[i,j,:],Ilemeanloc[i,k,:]))
                plt.plot(tmp[:,0],tmp[:,1],lw=tmplw[j,k],c=ReddishPurple)
            
    plt.plot(L1dihed[0:nmax:cut],L2dihed[0:nmax:cut],marker='.',markersize=2,lw=0,c='black')

    plt.xlim(0,360)
    plt.ylim(0,360)

    plt.xlabel(r'$\chi_1$ ($^{\circ}$)',fontsize=14)
    plt.ylabel(r'$\chi_2$ ($^{\circ}$)',fontsize=14)
    plt.text(25,325,valname[0:-5],fontsize=12)

plt.tight_layout()

plt.show()

In [ ]:
nchi = len(IleChiNames)
IleChiNames = RNHchiname[Ilechi1pointer]

npoints = len(RNHchiOpls4[:,Ilechi1pointer[0]])
tdelta = 2e-6/npoints*1e9
print('N samples',npoints,'Delta t',tdelta,'ns')

IleChiNames = RNHchiname[Ilechi1pointer]

fig = plt.figure(figsize=(8,nchi*3))

for i,val in enumerate(IleChiNames):  
    L1dihed = RNHchiOpls4[:,Ilechi1pointer[i]]
    L2dihed = RNHchiOpls4[:,Ilechi2pointer[i]]
    corrdim = 10000
    tdim = np.linspace(0,(corrdim-1)*tdelta,corrdim)

    j = 2*i+1
    plt.subplot(nchi,2,j)
    
    acf1 = DihedralCrossCorr(L1dihed*np.pi/180,L2dihed*np.pi/180,corrdim)
    acf2 = DihedralCrossCorr(L2dihed*np.pi/180,L1dihed*np.pi/180,corrdim)
    acf = (acf1+acf2)/2
    
    plt.plot(tdim,acf,c='black')
    plt.title(val[0:-5])
    plt.xlim(0,tdim[-1])
    #plt.axhline(0,c=Blue)
    plt.xlabel(r'$\tau$ (ns)',fontsize=14)
    plt.ylabel(r'$C(\tau)$',fontsize=14)

    plt.subplot(nchi,2,j+1)
    
    acf = DihedralCrossCorr(L1dihed*np.pi/180,L2dihed*np.pi/180,corrdim)
    
    plt.plot(tdim,acf,c='black')
    plt.title(val[0:-5])
    plt.xlim(0,10)
    #plt.axhline(0,c=Blue)
    plt.xlabel(r'$\tau$ (ns)',fontsize=14)

plt.tight_layout()
plt.show()

In [ ]:
statelabels = np.array(["mm","mt","mp","tm","tt","tp","pm","pt","pp"])

statepointer = np.array([[300,300],[300,180],[300,60],
                      [180,300],[180,180],[180,60],
                       [60,300],[60,180],[60,60]])

cut=50
npoints = np.shape(RNHchiOpls4)[0]/cut
tdelta = 2e-6/npoints*1e9
print('N samples',int(npoints),'Delta t',tdelta,'ns')

ind = 0
for ind,iname in enumerate(IleChiNames):
    print("\n",IleChiNames[ind])

    print("S2",Chi12S2rotamer12ave*IleS2jump12[ind]*RNHS2CAopls4[IleCD1pointer[ind]],
          "S2CA",RNHS2CAopls4[IleCD1pointer[ind]],
          "S2Jump",IleS2jump12[ind],"\n")
      
    for i in range(0,9):
        print(statelabels[i],np.round(Ilepopchi12[ind,i],3),end=" ")

    print("\n")

    for i in range(0,9):
        for j in range(0,9):
            if Iletransprob[ind,i,j] > 1:
                print(statelabels[i],statelabels[j],Iletransprob[ind,i,j])
            


In [ ]:
fig=plt.figure(figsize=(8,4))

plt.subplot(121,aspect=1)

i=0

L1dihed = RNHchiOpls4[:,Leuchi1pointer[i]].copy()
L2dihed = RNHchiOpls4[:,Leuchi2pointer[i]].copy()

for j,val in enumerate(L1dihed):
    if val < 0:
        L1dihed[j] = val + 360
    if L2dihed[j] < 0:
        L2dihed[j] = L2dihed[j] + 360

nmax = len(L1dihed)
ncut = 50

#plt.plot(L1dihed[0:nmax:ncut*10],L2dihed[0:nmax:ncut*10],lw=1,c=ReddishPurple)
plt.axhline(120,color=Blue,ls="--",lw=1)
plt.axhline(240,color=Blue,ls="--",lw=1)
plt.axvline(120,color=Blue,ls="--",lw=1)
plt.axvline(240,color=Blue,ls="--",lw=1)

tmpprobs = (Leutransprob[i,:,:] + np.transpose(Leutransprob[i,:,:]))/2.0

for j in range(0,8):
    for k in range(j,9):
        if tmpprobs[j,k] > 0:
            tmplw = 3*np.log10(np.maximum(1,100*tmpprobs/np.max(tmpprobs)))
            tmp = np.vstack((Leumeanloc[i,j,:],Leumeanloc[i,k,:]))
            plt.plot(tmp[:,0],tmp[:,1],lw=tmplw[j,k],c=ReddishPurple)
            
plt.plot(L1dihed[0:nmax:cut],L2dihed[0:nmax:cut],marker='.',markersize=2,lw=0,c='black')

plt.xlim(0,360)
plt.ylim(0,360)

plt.xlabel(r'$\chi_1$ ($^{\circ}$)',fontsize=14)
plt.ylabel(r'$\chi_2$ ($^{\circ}$)',fontsize=14)
plt.text(25,325,"(a)",fontsize=12)

plt.subplot(122,aspect=1)

i=6
L1dihed = RNHchiOpls4[:,Ilechi1pointer[i]].copy()
L2dihed = RNHchiOpls4[:,Ilechi2pointer[i]].copy()

for j,val in enumerate(L1dihed):
    if val < 0:
        L1dihed[j] = val + 360
    if L2dihed[j] < 0:
        L2dihed[j] = L2dihed[j] + 360
        
#plt.plot(L1dihed[0:nmax:ncut*10],L2dihed[0:nmax:ncut*10],lw=1,c=ReddishPurple)

plt.xlim(0,360)
plt.ylim(0,360)

plt.axhline(120,color=Blue,ls="--",lw=1)
plt.axhline(240,color=Blue,ls="--",lw=1)
plt.axvline(120,color=Blue,ls="--",lw=1)
plt.axvline(240,color=Blue,ls="--",lw=1)

tmpprobs = (Iletransprob[i,:,:] + np.transpose(Iletransprob[i,:,:]))/2.0

for j in range(0,8):
    for k in range(j,9):
        if tmpprobs[j,k] > 0:
            tmplw = 3*np.log10(np.maximum(1,100*tmpprobs/np.max(tmpprobs)))
            tmp = np.vstack((Ilemeanloc[i,j,:],Ilemeanloc[i,k,:]))
            plt.plot(tmp[:,0],tmp[:,1],lw=tmplw[j,k],c=ReddishPurple)
            
plt.plot(L1dihed[0:nmax:ncut],L2dihed[0:nmax:ncut],marker='.',markersize=2,lw=0,c='black')

plt.xlabel(r'$\chi_1$ ($^{\circ}$)',fontsize=14)
#plt.ylabel(r'$\chi_2$ ($^{\circ}$)',fontsize=14)
plt.text(25,325,"(b)",fontsize=12)

plt.tight_layout()

#plt.savefig("Figure7_Leu2_Ile116_chiplots.png",dpi=300,pad_inches=0)
#plt.savefig("Figure7_Leu2_Ile116_chiplots.eps",pad_inches=0)

plt.show()


In [ ]:
plt.close()